In [1]:
import pandas as pd
import datetime as date
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")

In [2]:
df = pd.read_csv("/Users/fernandoseguias/Desktop/ferseg/Proyectos/Programacion/iron_hack/project_1/output/cryptos_merged.csv")

#add year/month column to filter easier
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month

#substitude zero values in market_cap column for NaN
df.loc[df["market_cap"] == 0,"market_cap"] = np.nan

#replace price (ohlc), volume and currency null values
mask_price = df["open"].isnull(), ["open", "high", "low", "close"]
df.loc[mask_price] = df["price"]

mask_vol = df["volume"].isnull(), "volume"
df.loc[mask_vol] = df["total_volume"]

mask_cur = df["currency"].isnull(), "currency"
df.loc[mask_cur] = "USD"

mask_price1 = df["price"].isnull(), "price"
df.loc[mask_price1] = df["close"]

mask_vol1 = df["total_volume"].isnull(), "total_volume"
df.loc[mask_vol1] = df["volume"]

#df filter by date (scope)
df.set_index("year", inplace=True)
scope = df.loc[[2018, 2019, 2020, 2021], :]
scope.reset_index()

#drop duplicated columns
scope.drop(["currency", "price", "total_volume"], inplace=True, axis=1)

#supply column (market_cap = close * circulating_supply / circulating_supply = market_cap / close)
scope["circulating_supply"] = scope["market_cap"] / scope["close"]

#order columns
scope = scope.reset_index()
scope = scope[['coin', 'date', 'year', 'month', 'open', 'high', 'low', 'close', 'volume', 'market_cap', 'circulating_supply']]

#export clean csv
output_path = r"/Users/fernandoseguias/Desktop/ferseg/Proyectos/Programacion/iron_hack/project_1/output/scope.csv"
scope.to_csv(output_path, index = False)

scope

,coin,date,year,month,open,high,low,close,volume,market_cap,circulating_supply
0,btc,01/01/2018,2018,1,13850.50000,13921.50000,12877.70000,13444.90000,7.842500e+04,2.364403e+11,1.758588e+07
1,btc,02/01/2018,2018,2,13444.90000,15306.10000,12934.20000,14754.10000,1.377320e+05,2.570796e+11,1.742428e+07
2,btc,03/01/2018,2018,3,14754.10000,15435.00000,14579.70000,15156.60000,1.065430e+05,2.615052e+11,1.725355e+07
3,btc,04/01/2018,2018,4,15156.50000,15408.70000,14244.70000,15180.10000,1.109690e+05,2.681247e+11,1.766291e+07
4,btc,05/01/2018,2018,5,15180.10000,17126.90000,14832.40000,16954.80000,1.419600e+05,3.077740e+11,1.815262e+07
...,...,...,...,...,...,...,...,...,...,...,...
21360,atom,29/12/2021,2021,12,26.77838,26.77838,26.77838,26.77838,7.998134e+08,7.658409e+09,2.859922e+08
21361,atom,30/12/2021,2021,12,27.99376,27.99376,27.99376,27.99376,7.885243e+08,7.734245e+09,2.762846e+08
21362,ftm,27/07/2021,2021,7,0.20956,0.20956,0.20956,0.20956,7.798620e+07,5.388942e+08,2.571551e+09
21363,ftm,10/09/2021,2021,10,1.65739,1.65739,1.65739,1.65739,1.817521e+09,4.214438e+09,2.542816e+09


In [3]:
#dictionary creation dict = {coin type(str) : data per coin type(df)}
tickers = scope["coin"].unique()

dict_tickers = {}

for key in tickers.tolist():
    dict_tickers[key] = key
    dict_tickers[key] = scope.loc[scope.coin == key]

dict_tickers["btc"] #example value for key btc(bitcoin)

,coin,date,year,month,open,high,low,close,volume,market_cap,circulating_supply
0,btc,01/01/2018,2018,1,13850.5,13921.5,12877.7,13444.9,78425.0,2.364403e+11,1.758588e+07
1,btc,02/01/2018,2018,2,13444.9,15306.1,12934.2,14754.1,137732.0,2.570796e+11,1.742428e+07
2,btc,03/01/2018,2018,3,14754.1,15435.0,14579.7,15156.6,106543.0,2.615052e+11,1.725355e+07
3,btc,04/01/2018,2018,4,15156.5,15408.7,14244.7,15180.1,110969.0,2.681247e+11,1.766291e+07
4,btc,05/01/2018,2018,5,15180.1,17126.9,14832.4,16954.8,141960.0,3.077740e+11,1.815262e+07
...,...,...,...,...,...,...,...,...,...,...,...
14426,btc,27/12/2021,2021,12,50783.6,52016.3,50459.0,50678.2,43902.0,9.617239e+11,1.897707e+07
14427,btc,28/12/2021,2021,12,50703.4,50703.8,47345.7,47545.2,74391.0,9.602807e+11,2.019722e+07
14428,btc,29/12/2021,2021,12,47548.4,48121.7,46127.8,46461.7,63923.0,9.026558e+11,1.942796e+07
14429,btc,30/12/2021,2021,12,46470.7,47901.4,46003.0,47123.3,60960.0,8.743141e+11,1.855375e+07
